In [1]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

### Instantiate the video game reviews task with a randomly chosen seed

In [2]:
seed = np.random.randint(2**32 - 1)

task = VideogameReviewsTask(seed=seed)

### Create a tfdv schema by first auto-infering it from training data and then adjusting it

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

In [4]:
schema

feature {
  name: "marketplace"
  type: BYTES
  domain: "marketplace"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "customer_id"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "review_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_parent"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_title"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_category"
  type: BYT

### Train the baseline model for the task

In [5]:
model = task.fit_baseline_model()

### Run a stress test for the schema and the model with 250 randomly chosen corruptions, and mark performance drops of 3% as failures

In [6]:
stress_test = SchemaStresstest()
results = stress_test.run(task, model, schema, num_corruptions=250, performance_threshold=.03)

BrokenCharacters: {'column': 'product_id', 'fraction': 0.5}
MissingValues: {'column': 'customer_id', 'fraction': 0.48, 'sampling': 'MAR', 'na_value': ''}
MissingValues: {'column': 'product_parent', 'fraction': 0.67, 'sampling': 'MCAR', 'na_value': ''}
MissingValues: {'column': 'review_body', 'fraction': 0.03, 'sampling': 'MNAR', 'na_value': ''}
MissingValues: {'column': 'product_title', 'fraction': 0.08, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column': 'review_headline', 'fraction': 0.94}
BrokenCharacters: {'column': 'product_id', 'fraction': 0.56}
MissingValues: {'column': 'verified_purchase', 'fraction': 0.88, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.74}
MissingValues: {'column': 'product_title', 'fraction': 0.87, 'sampling': 'MNAR', 'na_value': ''}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.67}
SwappedValues: {'column': 'verified_purchase', 'fraction': 0.79, 'sampling': 'CAR', 'swap_with': 'verified_

### Look at the dataframe containing the results

In [7]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,"BrokenCharacters: {'column': 'product_id', 'fr...",TN,{},0.793205,0.793205
1,"MissingValues: {'column': 'customer_id', 'frac...",FP,"{'customer_id': description: ""String values th...",0.793205,0.793205
2,"MissingValues: {'column': 'product_parent', 'f...",FP,"{'product_parent': description: ""String values...",0.793205,0.793205
3,"MissingValues: {'column': 'review_body', 'frac...",TN,{},0.793205,0.793205
4,"MissingValues: {'column': 'product_title', 'fr...",TN,{},0.793205,0.793205
...,...,...,...,...,...
245,"MissingValues: {'column': 'review_body', 'frac...",TN,{},0.793205,0.793205
246,"Scaling: {'column': 'star_rating', 'fraction':...",FN,{},0.793205,0.597127
247,MissingValues: {'column': 'title_and_review_te...,TN,{},0.793205,0.771675
248,"BrokenCharacters: {'column': 'product_title', ...",TN,{},0.793205,0.793205
